In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [ ]:
##################################################
##THIS NOTEBOOK FOCUSES ON THE 'INNER JOIN' CLAUSE
##################################################
##I WILL START BY WRITING A QUERY THAT GATHERS DATA
##ABOUT THE INVIOCE OF ID =4
##################################################

In [6]:
%%sql
SELECT 
    il.track_id,
    t.name track_name,
    m.name track_type,
    t.unit_price,
    il.quantity
 FROM invoice_line il
 INNER JOIN track t ON t.track_id = il.track_id
 INNER JOIN media_type m ON m.media_type_id = t.media_type_id
 
 WHERE invoice_id = 4;

 * sqlite:///chinook.db
Done.


track_id,track_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",Protected AAC audio file,0.99,1
2560,Violent Pornography,MPEG audio file,0.99,1
3336,War Pigs,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,MPEG audio file,0.99,1
1872,Attitude,MPEG audio file,0.99,1
748,Dealer,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
2514,Spoonman,MPEG audio file,0.99,1


In [7]:
#################################################
## IN THE BELOW QUERY, I WILL ADD ONE MORE COLUMN
#################################################

In [8]:
%%sql
SELECT 
    il.track_id,
    t.name track_name,
    a.name artist_name,
    m.name track_type,
    il.unit_price,
    il.quantity
    
 FROM invoice_line il
 INNER JOIN track t ON t.track_id = il.track_id
 INNER JOIN media_type m ON m.media_type_id = t.media_type_id
 INNER JOIN album al ON al.album_id = t.album_id
 INNER JOIN artist a ON a.artist_id = al.artist_id
 WHERE il.invoice_id = 4;

 * sqlite:///chinook.db
Done.


track_id,track_name,artist_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",The King's Singers,Protected AAC audio file,0.99,1
2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
1872,Attitude,Metallica,MPEG audio file,0.99,1
748,Dealer,Deep Purple,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


In [9]:
##########################################
## BELOW QUERY LISTS THE TOP 10 ARTISTS
## CALCULATED BY THE TIMES A TRACK BY THAT
## ARTISTS HAS BEEN PURCHASED
##########################################

In [10]:
%%sql
SELECT
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC LIMIT 10;

 * sqlite:///chinook.db
Done.


artist,tracks_purchased
Queen,192
Jimi Hendrix,187
Nirvana,130
Red Hot Chili Peppers,130
Pearl Jam,129
AC/DC,124
Guns N' Roses,124
Foo Fighters,121
The Rolling Stones,117
Metallica,106


In [11]:
###################################
## BELOW IS A QUERY THAT RETURNS
## TEH TOP 5 ALBUMS, CALCULATED BY
## TIMES A TRACK FROM THE ALBUM HAS
## BEEN PURCHASED ~ MOST TO LEAST
###################################

In [12]:
%%sql
SELECT
    ta.album_title album,
    ta.artist_name artist,
    COUNT(*) tracks_purchased
  FROM invoice_line il
  INNER JOIN(


            SELECT t.track_id,
                al.title album_title,
                ar.name artist_name
                FROM track t
             INNER JOIN album al ON al.album_id = t.album_id
             INNER JOIN artist ar ON ar.artist_id = al.artist_id
      ) ta
      ON ta.track_id = il.track_id
GROUP BY 1, 2
ORDER BY 3 DESC Limit 5;


 * sqlite:///chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


In [13]:
#############################################
## BELOW IS A QUERY THAT RETURNS INFO
## ABOUT EACH EMPLOYEE & SUPERVISOR
## REGARDLESS IF THEY DON'T REPORT TO ANYONE
## USING || " " || TO CONCATNATE 2 COLUMNS
#############################################

In [14]:
%%sql
SELECT
    e1.first_name || " " || e1.last_name employee_name,
    e1.title employee_title,
    e2.first_name || " " || e2.last_name supervisor_name,
    e2.title supervisor_title
    FROM employee e1
    LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
    ORDER BY 1;

 * sqlite:///chinook.db
Done.


employee_name,employee_title,supervisor_name,supervisor_title
Andrew Adams,General Manager,None,None
Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
Laura Callahan,IT Staff,Michael Mitchell,IT Manager
Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
Michael Mitchell,IT Manager,Andrew Adams,General Manager
Nancy Edwards,Sales Manager,Andrew Adams,General Manager
Robert King,IT Staff,Michael Mitchell,IT Manager
Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


In [15]:
###################################################
## USING 'LIKE' (CASE INSENSITIVE) WHEN LOOKING FOR
## A PATTERN IN THE TABLE
## '%ABC' -> MATCHES ABC AT THE END OF STRING
## 'ABC%' -> MATCHES ABC AT THE START
## '%ABC%' -> MATCHES ABC ANYWHERE WITHIN STRING
## HOWEVER, IF LOOKING FOR A CASE INSENISTIVE MATCH,
## USE LOWER('[PATTERN]') TO EXCLUDE ANY CAPTIAL CASE
###################################################

In [ ]:
##########################################
## BELOW IS EXAMPLE OF THE 'LIKE' CLAUSE
##########################################

In [16]:
%%sql
SELECT first_name,
        last_name,
        phone
        FROM customer
        where first_name LIKE ("%belle%");

 * sqlite:///chinook.db
Done.


first_name,last_name,phone
Isabelle,Mercier,+33 03 80 73 66 99


In [17]:
########################################
## BELOW IS EXAMPLE OF LOWER() FUNCTION
########################################

In [20]:
%%sql
SELECT first_name,
        last_name,
        phone
        FROM customer
        where first_name LIKE LOWER("%IS%");

 * sqlite:///chinook.db
Done.


first_name,last_name,phone
François,Tremblay,+1 (514) 721-4711
Isabelle,Mercier,+33 03 80 73 66 99
Ladislav,Kovács,None
Stanisław,Wójcik,+48 22 828 37 39
Luis,Rojas,+56 (0)2 635 4444


In [21]:
#####################################
## WE CAN USE 'CASE' TO ADD A COLUMN
## IN THE BELOW EXAMPLE, WE WANT TO
## RETURN A TABLE THAT SUMMERIZES
## CUSTOMERS PURCHASES CASTED INTO INT
## & CATEGORIZED 
#####################################

In [34]:
%%sql
SELECT c.first_name || " " || c.last_name customer_name,
COUNT (i.invoice_id) number_of_purchases,
       CAST(SUM(i.total) AS IINT) total_spent,
       CASE
       WHEN SUM(i.total) < 40 THEN 'small spender'
       WHEN SUM(i.total) > 100 THEN 'big spender'
       ELSE 'regular'
       END
       AS customer_category
 FROM invoice i
 INNER JOIN customer c ON i.customer_id = c.customer_id
 GROUP BY 1 ORDER BY  3 DESC;

 * sqlite:///chinook.db
Done.


customer_name,number_of_purchases,total_spent,customer_category
František Wichterlová,18,144,big spender
Helena Holý,12,128,big spender
Hugh O'Reilly,13,114,big spender
Manoj Pareek,13,111,big spender
Luís Gonçalves,13,108,big spender
Fernanda Ramos,15,106,big spender
João Fernandes,13,102,big spender
François Tremblay,9,99,regular
Wyatt Girard,11,99,regular
Enrique Muñoz,11,98,regular
